In [1]:
# Script Name: Option_1 (Arbitrage Bot - Auto Scanning with Position Scaling)
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo
import json
import os

# API Keys (Bybit and Bitget only for this trial)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_API_PASSWORD = 'BgSecure789Trade2025'

# --- CONFIG ---
SPREAD_THRESHOLD = 0.8 / 100       # Minimum net spread to open a position
ADD_POSITION_USDT = 0             # Add-on position size in USDT
TRADE_USDT_SIZE = 30               # Initial trade size in USDT
MIN_VOLUME = 50000                 # Minimum token 24h quote volume
CHECK_INTERVAL = 1                 # Seconds between scans
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADE_FILE = 'open_trades.json'

TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
futures_markets_cache = {}
exchanges = {}

# --- INIT EXCHANGES ---
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'defaultType': 'spot'}
})
bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'defaultType': 'spot'}
})
exchanges = {'Bybit': bybit, 'Bitget': bitget}


In [ ]:
# --- JSON STORAGE ---
def load_open_trades():
    if os.path.exists(TRADE_FILE):
        with open(TRADE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_open_trades(trades):
    with open(TRADE_FILE, 'w') as f:
        json.dump(trades, f, indent=2)

open_trades = load_open_trades()

# --- HELPERS ---
def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last", None)
    except:
        return None

def get_best_bid_ask(exchange, symbol):
    order_book = exchange.fetch_order_book(symbol)
    best_bid = order_book['bids'][0][0] if order_book['bids'] else None
    best_ask = order_book['asks'][0][0] if order_book['asks'] else None
    return best_bid, best_ask

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    for s in futures_list:
        if s.startswith(base):
            return s
    return None

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable = []
        futures_markets_cache[exchange_name] = [s for s in markets if ":USDT" in s or "-USDT-SWAP" in s]
        for s, m in markets.items():
            if exchange_name == "Bitget" and not m.get('spot'): continue
            if '/USDT' in s and ':USDT' not in s and '-USDT-SWAP' not in s:
                try:
                    vol = exchange.fetch_ticker(s).get('quoteVolume', 0)
                    if vol >= MIN_VOLUME:
                        tradable.append(s)
                except: continue
        return tradable
    except: return []

# --- TRADE EXECUTION ---
def execute_trade(exchange_name, symbol, usdt_size):
    try:
        exchange = exchanges[exchange_name]
        spot_price = get_price(exchange, symbol)
        if not spot_price:
            print(f"❌ No spot price for {symbol}")
            return

        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)
        if not futures_symbol:
            print(f"❌ No futures match for {symbol}")
            return

        amount = usdt_size / spot_price
        best_bid, _ = get_best_bid_ask(exchange, symbol)
        order = exchange.create_order(symbol=symbol, type='limit', side='buy', amount=amount, price=best_bid)

        # Wait for fill
        wait_time, filled = 0, 0
        while wait_time < 15:
            current = exchange.fetch_order(order['id'], symbol)
            filled = current['filled']
            if filled > 0: break
            new_price = get_best_bid_ask(exchange, symbol)[0]
            if current['price'] != new_price:
                exchange.cancel_order(order['id'], symbol)
                order = exchange.create_order(symbol=symbol, type='limit', side='buy', amount=amount, price=new_price)
            time.sleep(1)
            wait_time += 1

        if filled == 0:
            exchange.cancel_order(order['id'], symbol)
            print(f"❌ Spot not filled: {symbol}")
            return

        # Futures execution
        if exchange_name == 'Bybit':
            fut = ccxt.bybit({'apiKey': BYBIT_API_KEY, 'secret': BYBIT_API_SECRET, 'options': {'defaultType': 'future'}})
        else:
            fut = ccxt.bitget({'apiKey': BITGET_API_KEY, 'secret': BITGET_API_SECRET, 'password': BITGET_API_PASSWORD, 'options': {'defaultType': 'swap'}})
        fut.set_leverage(1, futures_symbol)
        fut.create_order(symbol=futures_symbol, type='market', side='sell', amount=filled)

        # Save trade
        entry_spread = (get_price(fut, futures_symbol) - spot_price) / spot_price
        trade_id = f"{symbol.replace('/', '')}_{int(time.time())}"
        open_trades[trade_id] = {
            "symbol": symbol,
            "exchange": exchange_name,
            "amount": filled,
            "spot_price": spot_price,
            "futures_price": get_price(fut, futures_symbol),
            "entry_spread": entry_spread,
            "timestamp": datetime.datetime.now().isoformat()
        }
        save_open_trades(open_trades)
        print(f"✅ Executed & saved: {trade_id}")

    except Exception as e:
        print(f"❌ Execution failed: {e}")

# --- SCANNER LOGIC ---
def check_arbitrage(exchange, exchange_name, symbol):
    spot_price = get_price(exchange, symbol)
    futures_list = futures_markets_cache.get(exchange_name, [])
    futures_symbol = find_futures_symbol(symbol, futures_list)
    if not futures_symbol:
        return

    futures_price = get_price(exchange, futures_symbol)
    if not spot_price or not futures_price:
        return

    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (TRADING_FEES[exchange_name] * 2)

    for tid, trade in open_trades.items():
        if trade['symbol'] == symbol and trade['exchange'] == exchange_name:
            if spread - trade['entry_spread'] >= 0.005:
                print(f"🔁 Spread grew +0.5%, adding to {symbol}")
                execute_trade(exchange_name, symbol, ADD_POSITION_USDT)
            return

    if net_profit >= SPREAD_THRESHOLD:
        execute_trade(exchange_name, symbol, TRADE_USDT_SIZE)

def process_tokens(exchange, exchange_name, tokens):
    for token in tokens:
        check_arbitrage(exchange, exchange_name, token)

# --- MAIN LOOP ---
def main():
    while True:
        try:
            for name, ex in exchanges.items():
                tokens = get_tradable_tokens(ex, name)
                process_tokens(ex, name, tokens)
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Loop error: {e}")
            time.sleep(5)

if __name__ == '__main__':
    main()

❌ Execution failed: bybit fetchOrder() can only access an order if it is in last 500 orders(of any status) for your account. Set params["acknowledged"] = True to hide self warning. Alternatively, we suggest to use fetchOpenOrder or fetchClosedOrder
❌ Execution failed: bybit fetchOrder() can only access an order if it is in last 500 orders(of any status) for your account. Set params["acknowledged"] = True to hide self warning. Alternatively, we suggest to use fetchOpenOrder or fetchClosedOrder
❌ Execution failed: bybit fetchOrder() can only access an order if it is in last 500 orders(of any status) for your account. Set params["acknowledged"] = True to hide self warning. Alternatively, we suggest to use fetchOpenOrder or fetchClosedOrder
❌ Execution failed: bybit fetchOrder() can only access an order if it is in last 500 orders(of any status) for your account. Set params["acknowledged"] = True to hide self warning. Alternatively, we suggest to use fetchOpenOrder or fetchClosedOrder
❌ Ex